# Setup Ambiente di Lavoro

Prima di tutto, è **necessario** configurare l'ambiente di lavoro. La cella seguente clonerà il repository GitHub contenente tutti i moduli Python (`.py`) e si posizionerà nella directory corretta. 

**Esegui questa cella come primo passo.**

In [ ]:
# Clona il repository e posizionati nella directory corretta
# NOTA: Assicurati che l'URL del repository e il nome del branch siano corretti.
!git clone https://github.com/devedale/snn-ids.git snn-ids
%cd snn-ids
# Assicurati di usare il nome del branch corretto.
!git checkout feat/modular-ml-pipeline

# Manoscritto della Pipeline ML per Cybersecurity

Questo notebook è una guida interattiva e un manoscritto completo per una pipeline di machine learning **avanzata** per l'analisi di traffico di rete. È stato progettato con tre principi chiave in mente:

- **Modularità**: Ogni fase logica (preprocessing, training, predizione) è isolata nel proprio modulo Python (`.py`).
- **Configurabilità**: Tutta la pipeline è controllata da un singolo file, `config.py`.
- **Estensibilità**: La struttura è pensata per essere facilmente adattabile a nuovi dataset o a nuove architetture di modelli.

## 1. Architettura e Scelte Tecniche

La pipeline è composta dai seguenti elementi:

- `config.py`: Il cuore della configurazione. Qui si definiscono i percorsi, le colonne da usare e gli iperparametri per il training.
- `preprocessing/process.py`: Contiene tutta la logica per caricare i dati e trasformarli in un formato numerico che il modello può comprendere.
- `training/train.py`: Gestisce la creazione del modello, la grid search e il salvataggio del modello più performante.
- `prediction/predict.py`: Carica un modello salvato e lo usa per fare predizioni su nuovi dati.

## 2. Setup Iniziale

Installiamo le librerie necessarie.

In [ ]:
# Installa le dipendenze
!pip install pandas scikit-learn tensorflow faker

## 3. Download ed Estrazione del Dataset

La cella seguente si occupa di scaricare il dataset zippato da Kaggle e di estrarlo nella directory `data/`. Questo passaggio è fondamentale per avere i dati pronti per l'analisi.

**Nota**: Questo comando utilizza `curl` e `unzip`, che sono standard in ambienti Linux come Google Colab.

In [ ]:
!mkdir -p Downloads
!curl -L -o ./Downloads/improved-cicids2017-and-csecicids2018.zip \
  https://www.kaggle.com/api/v1/datasets/download/ernie55ernie/improved-cicids2017-and-csecicids2018

!unzip -o ./Downloads/improved-cicids2017-and-csecicids2018.zip -d ./data/

### Verifica dell'Estrazione

Eseguiamo un rapido controllo per assicurarci che i file siano stati estratti correttamente, visualizzando le prime righe di uno dei file CSV.

In [ ]:
!head -n 6 ./data/CSECICIDS2018_improved/Friday-02-03-2018.csv

## 4. Adattamento della Configurazione

Il file `config.py` è il cuore della nostra pipeline. Lo abbiamo già pre-configurato per funzionare con il nuovo dataset. Questa sezione spiega le modifiche chiave che sono state fatte. Puoi modificare `config.py` per sperimentare con diverse feature, iperparametri o strategie.

In particolare, puoi cambiare il `model_type` in `"gru"` per utilizzare la nuova architettura di rete neurale che abbiamo aggiunto.

## 5. Smoke Test: Esecuzione Rapida su un Campione

Questa sezione esegue l'intera pipeline su un piccolo sottoinsieme dei dati (le prime 10.000 righe). È un passaggio cruciale per verificare rapidamente che tutte le componenti (preprocessing, training, predizione) funzionino correttamente con la configurazione attuale, senza attendere i tempi di un training completo.

**Obiettivo**: Confermare la funzionalità end-to-end, non ottenere un modello performante.

In [ ]:
import time
from preprocessing.process import preprocess_data
from training.train import train_and_evaluate
from prediction.predict import predict_on_window
from config import PREPROCESSING_CONFIG

SMOKE_TEST_SAMPLE_SIZE = 10000

print(f"--- SMOKE TEST: Inizio dell'esecuzione su {SMOKE_TEST_SAMPLE_SIZE} campioni ---")
start_time = time.time()

# Fase 1: Preprocessing su un campione
print("\n--- FASE 1: Preprocessing ---")
X_sample, y_sample = preprocess_data(sample_size=SMOKE_TEST_SAMPLE_SIZE)

# Fase 2: Training su un campione
best_model_path_sample = None
if X_sample is not None and y_sample is not None:
    print("\n--- FASE 2: Training ---")
    _, best_model_path_sample = train_and_evaluate(X_sample, y_sample)
else:
    print("Dati di esempio non generati, salto il training.")

# Fase 3: Predizione di Esempio
print("\n--- FASE 3: Predizione ---")
if best_model_path_sample:
    window_size = PREPROCESSING_CONFIG.get('window_size', 10)
    # Creiamo una finestra di dati fittizi per la predizione
    sample_window = [
        {
            'Src IP': '10.0.0.1', 'Dst IP': '10.0.0.2', 'Src Port': 12345, 'Dst Port': 80, 'Protocol': 6,
            'Flow Duration': 1000, 'Total Fwd Packet': 5, 'Total Bwd packets': 3, 'Total Length of Fwd Packet': 200,
            'Total Length of Bwd Packet': 300, 'Flow Bytes/s': 500000, 'Flow Packets/s': 8000, 'Flow IAT Mean': 125.0,
            'Flow IAT Std': 50.0, 'Flow IAT Max': 200, 'Flow IAT Min': 100, 'Fwd IAT Mean': 250.0, 'Bwd IAT Mean': 333.0,
            'Fwd Header Length': 100, 'Bwd Header Length': 60, 'Average Packet Size': 62.5, 'Fwd Segment Size Avg': 40.0,
            'Bwd Segment Size Avg': 100.0
        } for _ in range(window_size)
    ]
    prediction_label = predict_on_window(sample_window, model_path=best_model_path_sample)
    if prediction_label:
        print(f"\nRISULTATO DELLO SMOKE TEST: La finestra di dati è stata classificata come: '{prediction_label}'")
else:
    print("Nessun modello addestrato nello smoke test, impossibile fare una predizione.")

end_time = time.time()
print(f"\n--- SMOKE TEST COMPLETATO in {end_time - start_time:.2f} secondi ---")

## 6. Esecuzione Completa: Training sul Dataset Intero

Questa è la sezione principale per addestrare il modello sull'intero dataset. A differenza dello smoke test, questa esecuzione utilizzerà tutti i dati disponibili per produrre un modello il più performante possibile.

**Attenzione**: L'esecuzione di questa cella richiederà una notevole quantità di tempo e risorse (RAM e CPU/GPU). Si consiglia di eseguirla in un ambiente con GPU attivata (come Google Colab Pro) e di essere pronti a un'attesa significativa.

Puoi modificare gli iperparametri nel file `config.py` per bilanciare tempo di training e performance.

In [ ]:
import time
from preprocessing.process import preprocess_data
from training.train import train_and_evaluate

print("--- ESECUZIONE COMPLETA: Inizio del training sul dataset intero ---")
start_time_full = time.time()

# Fase 1: Preprocessing completo
print("\n--- FASE 1: Preprocessing (Dataset Completo) ---")
# Chiamiamo preprocess_data() senza sample_size per usare tutti i dati
X_full, y_full = preprocess_data()

# Fase 2: Training completo
if X_full is not None and y_full is not None:
    print("\n--- FASE 2: Training (Dataset Completo) ---")
    train_and_evaluate(X_full, y_full)
else:
    print("Errore nel preprocessing dei dati completi, training annullato.")

end_time_full = time.time()
print(f"\n--- ESECUZIONE COMPLETA TERMINATA in {(end_time_full - start_time_full) / 60:.2f} minuti ---")

## 7. Conclusioni e Prossimi Passi

Questo notebook ha fornito una struttura completa e modulare per affrontare un problema di classificazione del traffico di rete utilizzando un dataset reale e complesso. Abbiamo visto come:

1.  **Impostare l'ambiente** e scaricare un dataset di grandi dimensioni.
2.  **Adattare la configurazione** della pipeline (`config.py`) per lavorare con le nuove feature.
3.  **Eseguire uno "Smoke Test"** per verificare rapidamente la funzionalità end-to-end.
4.  **Lanciare un training completo** per costruire un modello performante.

### Prossimi Passi

Da qui, ci sono molte direzioni interessanti da esplorare:

-   **Sperimentazione con le Feature**: Prova a modificare la lista `feature_columns` in `config.py`. Aggiungere o rimuovere feature può avere un impatto significativo sulle performance.
-   **Ottimizzazione degli Iperparametri**: Esegui una Grid Search più ampia modificando il dizionario `hyperparameters` in `config.py`. Prova diversi tassi di apprendimento, dimensioni dei batch o architetture di rete.
-   **Analisi degli Errori**: Esamina i casi in cui il modello sbaglia le previsioni. Questo può rivelare pattern specifici che il modello fatica a riconoscere e suggerire nuove feature da creare.
-   **Ricerca Accademica**: Come discusso, il prossimo passo potrebbe essere quello di cercare pubblicazioni che usano questo dataset e tentare di replicare le loro metodologie per confrontare i risultati.

Buon lavoro!